# Neural Attention: Batch Processing on ImageNet (Using VGG 1K)


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import transform, filters
import sys, pylab, operator, csv
import util
import util1
import os
import urllib
%matplotlib inline

caffe_root = '../'  # this file is expected to be in {caffe_root}/examples
sys.path.insert(0, caffe_root + 'python')

import caffe

We first load the model files and set the top layers' name.

In [ ]:

caffe.set_mode_gpu()
net = caffe.Net('../models/VGG16/VGG_ILSVRC_16_layers_deploy_bp.prototxt',
                '../models/VGG16/VGG_ILSVRC_16_layers_bp.caffemodel',
                caffe.TRAIN)
tags,tag2ID = util.loadTags('../data/ilsvrc12/synset_words.txt')
param = dict()
imgScale = 224
topBlobName = 'fc8_bp'
topLayerName = 'fc8_bp'
fwdLayerName = 'prob'
secondTopBlobName = 'fc7_bp'
secondTopLayerName = 'fc7_bp'


We will find a directory and iterate through all images in that directory

In [ ]:
#imagedirs = '/home/jinl/data/Clickture-ChallengeDataSet/Dev'
#destdirs = '/home/jinl/attention/GoogleNet/Clickture-ChallengeDataSet/Dev'

imagedirs = '/data1/imagenet-data/raw-data/validation'
destdirs = '/home/jinl/attention/VGG16/imagenet-data/raw-data/validation'

def createDirIfNotExist(dirn):
    if not os.path.exists(dirn):
        subpath = os.path.dirname(dirn)
        createDirIfNotExist(subpath)
        print "Create Directory", dirn
        os.mkdir(dirn)
    

pylab.rcParams['figure.figsize'] = (6.0, 6.0)

alldirectories = os.listdir(imagedirs)
for subdir in alldirectories:
    imagedir = imagedirs + '/' + subdir
    destdir = destdirs + '/' + subdir
    print "Working on Directory ", imagedir, "-->", destdir
    createDirIfNotExist(destdir)
    allfiles = os.listdir(imagedir)
    for image in allfiles:
        imgName = imagedir + '/' + image
        img = caffe.io.load_image(imgName)
        #print imgName
        minDim = min(img.shape[:2])
        # newSize = (int(img.shape[0]*imgScale/float(minDim)), int(img.shape[1]*imgScale/float(minDim)))
        newSize = (imgScale, imgScale);
        imgS = transform.resize(img, newSize)

        net.blobs['data'].reshape(1,3,newSize[0],newSize[1])
        transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
        transformer.set_mean('data', np.array([103.939, 116.779, 123.68]))
        transformer.set_transpose('data', (2,0,1))
        transformer.set_channel_swap('data', (2,1,0))
        transformer.set_raw_scale('data', 255.0)

        net.blobs['data'].data[...] = transformer.preprocess('data', imgS)
        out = net.forward(end = fwdLayerName)

        scores = net.blobs[topBlobName].data[0].reshape((len(tags),-1)).max(1).flatten() # pre-softmax scores
        tagScore = util.getTagScore(scores, tags, tag2ID)
        tagScore.sort(key = operator.itemgetter(1), reverse = True)
        topScores = tagScore[:5]
        print(topScores)


        def getTag(pair):
        #    print pair
            return pair[0]
        
        def getTagNameAndScore(pair):
            return pair[0]+'('+str(pair[1])+')'
        
        # specify the tags
        tagName = map( getTag ,topScores)
        #print(tagName)
        
        tagNameWithScore = map(getTagNameAndScore, topScores)

        # switch to the excitation backprop mode
        caffe.set_mode_eb_gpu() 
        #specify the output layer name
        outputLayerName = 'pool4'#'inception_5b/output'
        outputBlobName = 'pool4'#'inception_5b/output'

        attMaps = []

        for i in range(len(tagName)):
            tagID = tag2ID[tagName[i]]
            net.blobs[topBlobName].diff[0][...] = 0
            net.blobs[topBlobName].diff[0][tagID] = np.exp(net.blobs[topBlobName].data[0][tagID].copy())
            net.blobs[topBlobName].diff[0][tagID] /= net.blobs[topBlobName].diff[0][tagID].sum()
            
            # invert the top layer weights
            net.params[topLayerName][0].data[...] *= -1
            out = net.backward(start = topLayerName, end = secondTopLayerName)
            buff = net.blobs[secondTopBlobName].diff.copy()

            # invert back
            net.params[topLayerName][0].data[...] *= -1 
            out = net.backward(start = topLayerName, end = secondTopLayerName)

            # compute the contrastive signal
            net.blobs[secondTopBlobName].diff[...] -= buff
            
            out = net.backward(start = secondTopLayerName, end = outputLayerName)
            attMap = np.maximum(net.blobs[outputBlobName].diff[0].sum(0), 0)
            attMaps.append(attMap)
            
        saveImgName = destdir + '/' + image
        # util.showAttMap( img, attMaps, tagName, overlap = True, blur = False)
        util1.saveAttMap(img, attMaps, tagNameWithScore, saveTo=saveImgName, overlap = True, blur = False)
        # img.close()

Working on Directory  /data1/imagenet-data/raw-data/validation/n02483362 --> /home/jinl/attention/VGG16/imagenet-data/raw-data/validation/n02483362
[('n02483362 gibbon', 0.81378472), ('n02457408 three-toed sloth', 0.056800414), ('n02497673 Madagascar cat', 0.025854059), ('n02490219 marmoset', 0.025625717), ('n02500267 indri', 0.015745761)]


/usr/local/lib/python2.7/dist-packages/skimage/_shared/utils.py:174: skimage_deprecation: Mode 'nearest' has been renamed to 'edge'. Mode 'nearest' will be removed in a future release.
  "Mode 'nearest' has been renamed to 'edge'. Mode 'nearest' will be "


Save figure to: /home/jinl/attention/VGG16/imagenet-data/raw-data/validation/n02483362/ILSVRC2012_val_00028219.JPEG
[('n02483362 gibbon', 0.90853065), ('n02493793 spider monkey', 0.069210999), ('n02492660 howler monkey', 0.013351199), ('n02483708 siamang', 0.0057059228), ('n02480495 orangutan', 0.0015828676)]
Save figure to: /home/jinl/attention/VGG16/imagenet-data/raw-data/validation/n02483362/ILSVRC2012_val_00035799.JPEG
[('n02492660 howler monkey', 0.36771044), ('n02483362 gibbon', 0.29370219), ('n02483708 siamang', 0.18192679), ('n02480855 gorilla', 0.10413634), ('n02493509 titi', 0.036457952)]
Save figure to: /home/jinl/attention/VGG16/imagenet-data/raw-data/validation/n02483362/ILSVRC2012_val_00023595.JPEG
[('n02483362 gibbon', 0.99998307), ('n02483708 siamang', 7.2662688e-06), ('n02493793 spider monkey', 4.4005455e-06), ('n02488291 langur', 3.4058462e-06), ('n02492660 howler monkey', 1.1655483e-06)]
Save figure to: /home/jinl/attention/VGG16/imagenet-data/raw-data/validation/n02